# Test Set Bootstrapping

Once all the jobs of PREFER are completed and the session_xxx-xxx folders containing one folder results for each molecular representation have been created, one can evaluate the performances of the model on different bootstrapped datasets of the original test set. The performance of a simple Random Forest will be also collected on the same test data.

## Imports

In [ ]:
import sys
%load_ext autoreload
# path to the main directory
path_to_PREFER = 'path_to/PREFER/'
# path to submodules
path_to_cddd = 'path_to/model_based_representations/models/cddd/'
path_to_moler = 'path_to/model_based_representations/models/molecule-generation/'
sys.path.append(path_to_PREFER)
sys.path.append(path_to_cddd)
sys.path.append(path_to_moler)
import warnings
warnings.filterwarnings('ignore')
from prefer.utils.filtering import *
import sys

In [ ]:
from prefer.utils.post_processing_and_optimization_helpers import create_heat_map
from prefer.utils.automation import merge_table_metrics, data_preparation, generate_molecular_representations, run, create_comparison_table

### Set folders where the trained models are stored

In [ ]:
# The session_ folders contain one folder for each molecular representation
publiclogD = f'path_to/session_xxx-xxx'
publicsolubility = f'path_to/session_yyy-yyy'

problem_types = ['regression', 'classification']
assays = [publiclogD, publicsolubility]
assays_names = ['publicLogD',  'publicSolubility']
split_types = ['random',  'random']

In [ ]:
import numpy as np
import re
def convert(df, repr_name):
    import ast
    prova = df[repr_name].values
    X = []
    for index, elem in enumerate(prova):

        conv_elem = re.sub("\s+", ",", elem.strip())
        conv_elem = conv_elem.replace(',]', ']')
        conv_elem = conv_elem.replace('[,','[')
        conv = ast.literal_eval(conv_elem)
        X.append(conv)
    X = np.array(X)
    if 'Property_2' not in df.columns.values:
        y = df['Property_1'].to_numpy()
    else:  # multitasking
        if('Property_1' not in df):
            if('true_label_1' in df):
                df.rename(columns = {'true_label_1': 'Property_1'}, inplace = True)
            else:
                raise ValueError('Promens with columns')
            
        y = df[['Property_1', 'Property_2']].values
    return X, y

In [ ]:
import random
def bootstrap(X, y, seed, perc = 0.8):
    x = list(range(X.shape[0]))
    random.Random(seed).shuffle(x)
    limit = int(len(x) * perc)
    index = x[:limit]
    return X[index,:], y[index]

In [ ]:
def compute_deltaAUPRC(labels, predictions, y_train):
    from sklearn.metrics import precision_recall_curve, auc
    precision, recall, _ = precision_recall_curve(labels, predictions)
    auc_score = auc(recall, precision)
    deltaAUPRC = round(auc_score - (np.sum(y_train) / len(y_train)), 3)
    return deltaAUPRC

In [ ]:
def compute_roc_auc(labels, predictions):
    from sklearn.metrics import roc_auc_score
    ROC_AUC = round(roc_auc_score(labels, predictions), 3)
    return ROC_AUC

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

def baseline(problem_type, Xtrain, ytrain, Xtest, ytest):

    if problem_type == 'regression':
        metric = 'R2'
        rf = RandomForestRegressor(max_depth=20, n_estimators = 100, random_state=0)
        rf.fit(Xtrain, ytrain)
        predictions = rf.predict(Xtest)
    elif problem_type == 'classification':
        metric = 'kappa'
        rf = RandomForestClassifier(max_depth=20, n_estimators = 100, random_state=0)
        rf.fit(Xtrain, ytrain)
        predictions = rf.predict_proba(Xtest)[:, 1]
        #from sklearn.metrics import roc_auc_score
        #RF_baseline_performance = roc_auc_score(ytest, predictions)
    print(Xtest_new.shape)
    print(predictions.shape)
    return predictions

In [ ]:
def performances(ytest, predictions, ytrain, problem_type):
    from sklearn.metrics import r2_score
    num_tasks = ytest.ndim

    if problem_type == 'regression':
        if(num_tasks == 1):
            performance = r2_score(ytest, predictions)
        else:
            performance = []
            for task in range(0, num_tasks):
                performance.append(r2_score(ytest[:,task], predictions[:, task]))
    elif problem_type == 'classification':
        performance = compute_roc_auc(ytest, predictions)
    return performance
    

In [ ]:
from prefer.molecule_representations.fingerprints_representations_builder import (
    FingerprintsRepresentationsBuilder,
)

from prefer.molecule_representations.descriptors2D_representations_builder import (
    Descriptors2DRepresentationsBuilder,
)

def compute_representation_again(df, split_type, repr_name):
    df1 = df.copy()
    if split_type == 'temporal':
        df1['Time'] = pd.to_datetime(df1['Time'])
    if(repr_name == 'FINGERPRINTS'):
        df1 = df1.drop(columns = ['FINGERPRINTS', 'is_train'])
        fing_representation = FingerprintsRepresentationsBuilder()
        fingerprints = fing_representation.build_representations(df1, split_type=split_type)
        Xtrain, ytrain, Xtest, ytest, index_train, index_test = fingerprints.split(return_indices = True)
    elif(repr_name == 'DESCRIPTORS2D'):
        df1 = df1.drop(columns = ['DESCRIPTORS2D', 'is_train'])
        _2d_descriptors = Descriptors2DRepresentationsBuilder()
        _2dd = _2d_descriptors.build_representations(df1, split_type=split_type)
        Xtrain, ytrain, Xtest, ytest, index_train, index_test = _2dd.split(return_indices = True)
    else:
        raise ValueError(f'representation neither FINGERPRINTS neither 2DD')
        
    return Xtrain, ytrain, Xtest, ytest, index_train, index_test

In [ ]:
from prefer.utils.features_scaling import apply_scaling

In [ ]:
## Retrieve Bench object
import os
import pandas as pd
import pickle
autosklearn_performances = dict()
RF_performances = dict()
for problem_type,assay, assays_name, split_type in zip(problem_types, assays, assays_names, split_types):
    try:
        print(assay)
        print(assays_name)
        autosklearn_performances[assays_name] = dict()
        RF_performances[assays_name] = []
        repr_folders = os.listdir(assay)
        for repr_folder in repr_folders:
            if((not repr_folder.startswith('_')) and (not repr_folder.startswith('.'))):
                print(repr_folder)
                current_representation = repr_folder.split('_')[0]
                autosklearn_performances[assays_name][current_representation]=[]
                print(current_representation)
                path_bench = f'{assay}/{repr_folder}/bench.pkl'
                # import bench
                import pickle
                with open(path_bench, 'rb') as f:
                    bench = pickle.load(f)

                # take Xtest and ytest
                if('Property_2' in bench['df'][current_representation].columns.values):
                    if('true_label_1' in bench['df'][current_representation].columns.values):
                        bench['df'][current_representation].rename(columns={'true_label_1': 'Property_1'}, inplace = True)
                    ytest = bench['df'][current_representation][bench['df'][current_representation].is_train == False][['Property_1', 'Property_2']].values
                    ytrain = bench['df'][current_representation][bench['df'][current_representation].is_train == True][['Property_1', 'Property_2']].values
                else:
                    ytest = bench['df'][current_representation][bench['df'][current_representation].is_train == False].Property_1.values
                    ytrain = bench['df'][current_representation][bench['df'][current_representation].is_train == True].Property_1.values
                Xtest = np.stack(bench['df'][current_representation][bench['df'][current_representation].is_train == False][current_representation].tolist())
                Xtrain = np.stack(bench['df'][current_representation][bench['df'][current_representation].is_train == True][current_representation].tolist())
     
                featuriz = bench['features_scaling_type'][current_representation]
                print(f'>>>>>{featuriz}')
                if bench['features_scaling_type'][current_representation] is not None:
                    print('FEATURIZATION NEEDED')
                    # Scale features before prediction.
                    Xtest = [
                        apply_scaling(
                            features_vect=x,
                            scaling_type=bench['features_scaling_type'][current_representation],
                            means=bench['features_means_vect'][current_representation],
                            stds=bench['features_stds_vect'][current_representation],
                        )
                        for x in Xtest
                    ]
                    Xtest = np.array(Xtest)
                    # Scale features before prediction.
                    Xtrain = [
                        apply_scaling(
                            features_vect=x,
                            scaling_type=bench['features_scaling_type'][current_representation],
                            means=bench['features_means_vect'][current_representation],
                            stds=bench['features_stds_vect'][current_representation],
                        )
                        for x in Xtrain
                    ]
                    Xtrain = np.array(Xtrain)
                
                # bootstrap
                seeds = [0, 1, 2, 3, 4, 5]
                for seed in seeds:
                    print(f'Current seed: {seed}')
                    Xtest_new, ytest_new = bootstrap(Xtest, ytest, seed, perc = 0.8)
                    print(f'autosklearn_predictions')
                    if problem_type == 'classification':
                        autosklearn_predictions = bench['best_estimator'][current_representation].predict_proba(Xtest_new)[:, 1]
                    else:
                        autosklearn_predictions = bench['best_estimator'][current_representation].predict(Xtest_new)
                    print(f'RF_predictions')
                    RF_predictions = baseline(problem_type, Xtrain, ytrain, Xtest_new, ytest_new)
                    print(f'autosklearn_performances')
                    perf = performances(ytest_new, autosklearn_predictions, ytrain, problem_type)
                    autosklearn_performances[assays_name][current_representation].append(perf)
                    print(f'RF_performances')
                    perf = performances(ytest_new, RF_predictions, ytrain, problem_type)
                    RF_performances[assays_name].append(perf)
    except Exception as e:
        print(f'problem with {assays_name} in particular {e}')
        continue
 

In [ ]:
final_dict = dict()
final_dict['RF'] = RF_performances
final_dict['autosklearn'] = autosklearn_performances
name = str(assays_names)
with open(f'final_dict_{name}.pickle', 'wb') as handle:
    pickle.dump(final_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)